In [1]:
!wget -O trainingset.zip https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1

--2025-01-03 07:58:46--  https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813899321 (776M) [application/octet-stream]
Saving to: ‘trainingset.zip’

trainingset.zip     100%[===================>] 776.19M  10.9MB/s    in 75s     

2025-01-03 08:00:01 (10.4 MB/s) - ‘trainingset.zip’ saved [813899321/813899321]



In [2]:
!wget -O testset.zip https://zenodo.org/records/11061650/files/ideology-power-st-testset.zip?download=1

--2025-01-03 08:00:01--  https://zenodo.org/records/11061650/files/ideology-power-st-testset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 268140953 (256M) [application/octet-stream]
Saving to: ‘testset.zip’

testset.zip         100%[===================>] 255.72M  12.1MB/s    in 23s     

2025-01-03 08:00:25 (11.2 MB/s) - ‘testset.zip’ saved [268140953/268140953]



In [3]:
!unzip trainingset.zip -d trainingset


Archive:  trainingset.zip
   creating: trainingset/orientation/
  inflating: trainingset/orientation/orientation-at-train.tsv  
  inflating: trainingset/orientation/orientation-ba-train.tsv  
  inflating: trainingset/orientation/orientation-be-train.tsv  
  inflating: trainingset/orientation/orientation-bg-train.tsv  
  inflating: trainingset/orientation/orientation-cz-train.tsv  
  inflating: trainingset/orientation/orientation-dk-train.tsv  
  inflating: trainingset/orientation/orientation-ee-train.tsv  
  inflating: trainingset/orientation/orientation-es-ct-train.tsv  
  inflating: trainingset/orientation/orientation-es-ga-train.tsv  
  inflating: trainingset/orientation/orientation-es-train.tsv  
  inflating: trainingset/orientation/orientation-fi-train.tsv  
  inflating: trainingset/orientation/orientation-fr-train.tsv  
  inflating: trainingset/orientation/orientation-gb-train.tsv  
  inflating: trainingset/orientation/orientation-gr-train.tsv  
  inflating: trainingset/orientati

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv("./trainingset/orientation/orientation-tr-train.tsv", delimiter = "\t")
train_df, test_df = train_test_split(
    df, test_size=0.1, stratify=df['label'], random_state=42
)
print(train_df)


            id                           speaker sex  \
11334  tr11334  0e89ab0a8d7fd030dcde1ac4c9bc835b   M   
15381  tr15381  8bd5bf78e67110b62f2422d286833b9f   M   
5431   tr05431  be82a4ade406ec6774a0a2e38f6957e3   M   
9486   tr09486  853677cdc5a5b426ca8a79036210baaf   F   
3549   tr03549  98fe6edd8a618c1c26a0dad468666275   M   
...        ...                               ...  ..   
7019   tr07019  9ce564673a5182dd246cd60e2fbc2aec   M   
2247   tr02247  3a4409d1f226bd84d3a26813966de314   F   
8358   tr08358  66460cda11160d4044ff006d3d6bdf49   M   
5748   tr05748  e3b89cafbddc575e6cd369a0c34b8be4   F   
9264   tr09264  64288895a2343df765f7a30b39e17070   M   

                                                    text  \
11334  Teşekkür ederim Sayın Başkan. <p> Tarım Bakanı...   
15381  Sayın Başkan, değerli milletvekilleri; sizleri...   
5431   Değerli arkadaşlar, bir örnek daha vereceğim: ...   
9486   Ama sonuçta halkımız bunun kararını verecek. G...   
3549   Sayın Komisyon Başka

In [ ]:
from transformers import XLMRobertaForSequenceClassification, AutoTokenizer

mymodel_bert= "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(mymodel_bert)
model = XLMRobertaForSequenceClassification.from_pretrained(mymodel_bert, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_dataset(examples):
    return bert_tokenizer(examples["text_en"].tolist(), padding="max_length", truncation=True)

train_encodings = tokenize_dataset(train_df)
test_encodings = tokenize_dataset(test_df)


In [ ]:
import torch.nn as nn
classifier = nn.Linear(768, 2)


In [ ]:
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
train_inputs = [bert_tokenizer(text, padding="max_length", truncation=True, return_tensors="pt") for text in train_df['text_en']]
test_inputs = [bert_tokenizer(text, padding="max_length", truncation=True, return_tensors="pt") for text in test_df['text_en']]

train_dataset = torch.utils.data.TensorDataset(
    torch.cat([x['input_ids'] for x in train_inputs]),
    torch.cat([x['attention_mask'] for x in train_inputs]),
    torch.tensor(list(train_df['label']))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.cat([x['input_ids'] for x in test_inputs]),
    torch.cat([x['attention_mask'] for x in test_inputs]),
    torch.tensor(list(test_df['label']))
)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
from torch.optim import AdamW,SGD
from transformers import get_scheduler
num_epochs=3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, attention_mask, labels = batch
        batch = {}
        batch["input_ids"] = inputs.to(device)
        batch["attention_mask"] = attention_mask.to(device)
        batch["labels"]= labels.to(device)


        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1362 [00:00<?, ?it/s]

In [ ]:
!pip install evaluate
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in test_loader:
    inputs, attention_mask, labels = batch
    batch = {}
    batch["input_ids"] = inputs.to(device)
    batch["attention_mask"] = attention_mask.to(device)
    batch["labels"]= labels.to(device)

    b = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**b)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


{'accuracy': 0.8674101610904585}

In [ ]:
!pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import pandas as pd
from transformers import pipeline
from sklearn.model_selection import train_test_split

classifier = pipeline(
    "zero-shot-classification",
    model="bigscience/bloom-560m",
    device=0
)

labels = ["Label 0", "Label 1"]

test_texts = test_df["text_en"].tolist()[:10]  #for gpu constraints
results = classifier(test_texts, candidate_labels=labels)

for i, res in enumerate(results):
    print(f"Text: {test_texts[i]}")
    print(f"Prediction: {res['labels'][0]} with score {res['scores'][0]:.4f}")
    print(f"Prediction: {res['labels'][1]} with score {res['scores'][1]:.4f}")
    print("-" * 50)


Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Mr. President, I have made a commitment to the PTA's motion to investigate the motories' problems. I salute the General Assembly. <p> Dear lawmakers, the actions of thousands of Messepeti workers moto couriers are today on their 17th day. Our motorcade brothers, who were hosted under the roof of our House the other day, are being heavily exploited under heavy conditions. In pandemic conditions, they worked their lives off, and the companies they worked for made them worth a lot more than they were worth. But the motoriers are forced to work for the price of misery. In one night, thousands of motorists were stripped of their rights to join the union by changing their line of work. About 900,000 motories are fighting for their lives in our country. They're fighting for their lives because they're under heavy exploitation costs; they're fighting for their lives because they're being sent to die in traffic under early delivery pressure. In 2020, 280 motories died in car accidents, an

In [ ]:
torch.cuda.empty_cache()